In [ ]:
import pandas as pd
import numpy as np
import magic
from matplotlib import pyplot as plt
import importlib
%matplotlib inline
importlib.reload(magic)

In [ ]:
runner = magic.Fluent()
(runner
     .create_historical_players(3)
     .battling_in_pairs()
     .on_maps('sample.json','Sierpinski-triangle.json')
     .repeating(10)
     .experiment('Historical')
     #.preview()
     .run().dump().store_pointwise('historical.csv')
)
     

In [ ]:
runner.token

In [ ]:

data = pd.read_csv('historical.csv', index_col=False)

#data

In [ ]:
groups = data.groupby(['server_name','map']).mean()['scores'].to_frame().reset_index()
pd.pivot_table(groups, index='map',values='scores',columns='server_name')